In [1]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.50.0


In [7]:
!nvidia-smi

Tue Nov 25 16:21:15 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   82C    P0             204W / 300W |  64824MiB / 81920MiB |     94%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/workspace/Approach0


In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Name

In [21]:
dataset_name = "thenlpresearcher/flores22_eng_mar_hin"

In [10]:
from datasets import load_dataset
raw_datasets = load_dataset(dataset_name)

In [20]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["src"]
gt_mar     = raw_datasets['test']["mar_Deva"]
gt_hin    = raw_datasets['test']["hin_Deva"]

## Model Names

In [59]:
original_model_name = "ai4bharat/indictrans2-en-indic-dist-200M"
without_punct_model_name = "thenlpresearcher/iitb-en-indic-without-punct"
with_punct_model_name = "thenlpresearcher/iitb-en-indic-only-punct"
combined_x_punct_model_name = "thenlpresearcher/shalaka_fd_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"
combined_2x_punct_model_name = "thenlpresearcher/shalaka_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"

### Original

In [23]:
mode = "original"

In [13]:
model_name = original_model_name

In [14]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

2025-11-25 16:21:41.727726: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 16:21:41.790242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 16:21:43.908153: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|███████████████████████████████████████| 32/32 [01:07<00:00,  2.11s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी प्रमाणित इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते.
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: प्रमुख संशोधकांचे म्हणणे आहे की यामुळे कमी उत्पन्न असलेल्या देशांतील रुग्णांना कर्करोग, क्षयरोग, एच. आय. व्ही. आणि मलेरिया लवकर आढळू शकतो, जिथे स्तनाच्या कर्करोगासारख्या आजारांचे जगण्याचे प्रमाण श्रीमंत देशांच्या तुलनेत

In [16]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी प्रमाणित इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते.


In [18]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [00:32<00:00,  1.02s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: सोमवार को, स्टैनफोर्ड यूनिवर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो कोशिकाओं को प्रकार के अनुसार क्रमबद्ध कर सकता हैः एक छोटी छापने योग्य चिप जिसे संभवतः लगभग एक अमेरिकी प्रतिशत के लिए मानक इंकजेट प्रिंटर का उपयोग करके निर्मित किया जा सकता है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: प्रमुख शोधकर्ताओं का कहना है कि इससे कम आय वाले देशों में कैंसर, तपेदिक, एच. आई. वी. और मलेरिया का जल्द पता चल सकता है, जहां स्तन कैंसर जैसी बीमारियों के लिए जीवित रहने की द

In [19]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_hin = remove_prefix(translations, "hin_Deva eng_Latn ")
print(translations_hin[0])

सोमवार को, स्टैनफोर्ड यूनिवर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो कोशिकाओं को प्रकार के अनुसार क्रमबद्ध कर सकता हैः एक छोटी छापने योग्य चिप जिसे संभवतः लगभग एक अमेरिकी प्रतिशत के लिए मानक इंकजेट प्रिंटर का उपयोग करके निर्मित किया जा सकता है।


In [24]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/original_outputs.csv


In [33]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [34]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 18.687773491149727, CHRF++: 54.481308422070896
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695


### Without Punctuation Model

In [41]:
mode = "without"

In [35]:
model_name = without_punct_model_name

In [36]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [38]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [00:46<00:00,  1.47s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एक नवीन निदान साधनाचा शोध जाहीर केला जो प्रकारानुसार पेशींना क्रमवारी लावू शकतोः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटर वापरून तयार केली जाऊ शकते, शक्यतो प्रत्येकी सुमारे एक <ID1>.
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn आघाडीच्या संशोधकांचे म्हणणे आहे की यामुळे कमी उत्पन्न असलेल्या देशांमध्ये कर्करोग, क्षयरोग, एचआयव्ही आणि मलेरियाचे लवकर निदान होऊ शकते, जिथे स्तनाच्या कर्करोगासारख्या आज

In [39]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एक नवीन निदान साधनाचा शोध जाहीर केला जो प्रकारानुसार पेशींना क्रमवारी लावू शकतोः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटर वापरून तयार केली जाऊ शकते, शक्यतो प्रत्येकी सुमारे एक <ID1>.


In [42]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:04<00:00,  2.02s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवार को, स्टैनफोर्ड विश्वविद्यालय स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो प्रकार के अनुसार कोशिकाओं को वर्गीकृत कर सकता हैः एक छोटी छापने योग्य चिप, जो संभवतः एक अमेरिकी सेंट के लिए मानक इंकजेट प्रिंटर का उपयोग करके निर्मित की जा सकती है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn प्रमुख शोधकर्ताओं का कहना है कि कम आय वाले देशों में, जहां स्तन के कर्करोग जैसी रोगों के जीवित रहने की दर अमीर देशों की तुलना में आधी हो सकती है, 

In [43]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/without_outputs.csv


In [44]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [45]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 19.268959094639783, CHRF++: 54.72795569179818
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695


### With Punctuation Model

In [46]:
mode = "with"

In [47]:
model_name = with_punct_model_name

In [48]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [49]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [00:45<00:00,  1.42s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी नवीन निदानात्मक साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटर वापरून तयार केली जाऊ शकते, शक्यतो प्रत्येकी सुमारे एक अमेरिकन सेंटी.
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn आघाडीच्या संशोधकांचे म्हणणे आहे की यामुळे कमी उत्पन्न असलेल्या देशांमधील रुग्णांना कर्करोग, क्षयरोग, एचआयव्ही आणि मलेरिया लवकर आढळू शकतो, जिथे स्तनाच

In [50]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी नवीन निदानात्मक साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटर वापरून तयार केली जाऊ शकते, शक्यतो प्रत्येकी सुमारे एक अमेरिकन सेंटी.


In [51]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:11<00:00,  2.22s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवार को, स्टैनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो प्रकार द्वारा कोशिकाओं को वर्गीकृत कर सकता है: एक छोटी छापने योग्य चिप जो मानक इंकजेट प्रिंटर का उपयोग करके संभवतः एक अमेरिकी सेंट के लिए निर्मित की जा सकती है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn प्रमुख शोधकर्ताओं का कहना है कि इससे कम आय वाले देशों में कर्करोग, क्षयरोग, एच. आय. वी. और मलेरिया का शीघ्र निदान हो सकता है, जहां स्तन कर्करोग जैसी र

In [52]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})


import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/with_outputs.csv


In [53]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [54]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 19.200741993492343, CHRF++: 54.78083136206715
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695


### Combined Punctuation Model (2x)

In [60]:
mode = "combined_2x"

In [61]:
model_name = combined_2x_punct_model_name

In [62]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [63]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:39<00:00,  3.10s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%...................................................................................................................................................................................
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
ma

In [64]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%.


In [65]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:28<00:00,  2.78s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवार को, स्टैनफोर्ड यूनिवर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो प्रकार के अनुसार कोशिकाओं को क्रमबद्ध कर सकता हैः एक छोटी छापने योग्य चिप जो संभवतः एक अमेरिकी प्रतिशत के लिए मानक इंकजेट प्रिंटर का उपयोग करके निर्मित की जा सकती है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn प्रमुख शोधकर्ताओं का कहना है कि यह कम आय वाले देशों में कैंसर, तपेदिक, एचआईवी और मलेरिया का शीघ्र पता लगा सकता है, जहां स्तन कैंसर जैसी बीमारियों 

In [66]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/combined_2x_outputs.csv


In [67]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [68]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 19.371112617185684, CHRF++: 55.10930670014661
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695


### Combined Punctuation Model (x)

In [69]:
mode = "combined_x"

In [70]:
model_name = combined_x_punct_model_name

In [71]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [72]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [02:03<00:00,  3.85s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%...................................................................................................................................................................................
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
ma

In [73]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%.


In [74]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:49<00:00,  3.42s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवार को, स्टैनफोर्ड यूनिवर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो प्रकार के अनुसार कोशिकाओं को क्रमबद्ध कर सकता हैः एक छोटी छापने योग्य चिप जिसे मानक इंकजेट प्रिंटर का उपयोग करके संभवतः एक-एक अमेरिकी प्रतिशत के लिए निर्मित किया जा सकता है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn प्रमुख शोधकर्ताओं का कहना है कि इससे कम आय वाले देशों में कैंसर, तपेदिक, एचआईवी और मलेरिया का शीघ्र पता चल सकता है, जहां स्तन कैंसर जैसी बी

In [75]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/combined_x_outputs.csv


In [76]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [77]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 19.46068524510397, CHRF++: 55.10799795709722
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695


## T5

In [78]:
model_name = original_model_name
mode = "t5"

In [79]:
from transformers import pipeline

# Use batching + GPU (if available)
punctuator_pipeline = pipeline(
    "text2text-generation",
    model="thenlpresearcher/iitb-t5-finetuned-punctuation",
    device=device,            # GPU; use device=-1 for CPU
    batch_size=64        # adjust based on GPU RAM
)

def restore_punctuation_t5_batch(text_list):
    # The pipeline automatically batches under the hood
    outputs = punctuator_pipeline(
        text_list,
        max_length=128
    )
    # Pipeline returns list of dicts
    return [o["generated_text"] for o in outputs]

Device set to use cuda


In [80]:
# Run the whole batch in parallel
predicted_sentences = restore_punctuation_t5_batch(src_sentences)
print(len(predicted_sentences))

997


In [81]:
input_sentences = predicted_sentences

In [82]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [02:11<00:00,  4.11s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%...................................................................................................................................................................................
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
ma

In [83]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

सोमवारी, स्टॅनफोर्ड युनिव्हर्सिटी स्कूल ऑफ मेडिसिनच्या शास्त्रज्ञांनी एका नवीन निदान साधनाचा शोध लावल्याची घोषणा केली जी पेशींना प्रकारानुसार क्रमवारी लावू शकतेः एक लहान छापण्यायोग्य चिप जी मानक इंकजेट प्रिंटरचा वापर करून तयार केली जाऊ शकते, बहुधा प्रत्येकी सुमारे एक <ID1>.%.


In [84]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [01:56<00:00,  3.63s/it]

eng_Latn: On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.
mar_Deva: mar_Deva eng_Latn सोमवार को, स्टैनफोर्ड यूनिवर्सिटी स्कूल ऑफ मेडिसिन के वैज्ञानिकों ने एक नए नैदानिक उपकरण के आविष्कार की घोषणा की जो प्रकार के अनुसार कोशिकाओं को क्रमबद्ध कर सकता हैः एक छोटी छापने योग्य चिप जिसे मानक इंकजेट प्रिंटर का उपयोग करके संभवतः एक-एक अमेरिकी प्रतिशत के लिए निर्मित किया जा सकता है।
eng_Latn: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
mar_Deva: mar_Deva eng_Latn प्रमुख शोधकर्ताओं का कहना है कि इससे कम आय वाले देशों में कैंसर, तपेदिक, एचआईवी और मलेरिया का शीघ्र पता चल सकता है, जहां स्तन कैंसर जैसी बी

In [85]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/flores22_eng_mar_hin/t5_outputs.csv


In [86]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [87]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 19.335163203393773, CHRF++: 55.01783414973131
Metrics -- Hindi
BLEU: 36.38818669851306 CHRF++: 61.329541850065695
